In [2]:
import difflib
from functools import reduce
import pandas as pd

# 오타교정의 유용성
* 캐시 히트율이 올라감 -> 서버부하 적어짐
* 사용자 경험이 좋아짐

<img src="http://www.comworld.co.kr/news/photo/201511/48900_28534_336.jpg"/> 

# 유니코드와 인코딩

## 유니코드 - 글자와 코드가 1:1로 매핑된 시스템
    * 매핑된 표를 보통 유니코드 코드 포인트라고 부른다.
    * 유니코드 이전에는 코드페이지라고 불렀다. (CP949)
## 인코딩 - 코드페이지를 컴퓨터 메모리에 어떻게 표시할지 정의한 방식
    * UTF-8 - 영어는 1바이트로 표현하고 한글은 3바이트로 표현해야지
    * EUC-KR / CP-949 - 영어는 1바이트로 표현하고 한글은 2바이트로 표현해야지
## 조합형 / 완성형 인코딩 방식
    * 조합형 - 초중종성을 조합해서 한 글자를 표현하는 인코딩 방식 (거의 모든 글자 표현가능, 160만자 - 실제 유효한글은 <11700자)
    * 완성형 - 한글자가 하나의 글자 (표현할 수 있는 글자가 적음 - EUC-KR 은 2350자 표현가능)
## 유니코드로 한글 쓸땐 완성형 인코딩 방식이 표준
    * 완성된 글자 및 자/모음이 각각 코드페이지에 할당되어 있다.

# 유니코드 글자에서 자모음 분리하기

<img src="http://www.comworld.co.kr/news/photo/201511/48900_28535_337.jpg"/>

In [3]:
chut = 'ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ#'
ga = 'ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ#'
ggut = ' ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ#'

BASE = 0xAC00

In [4]:
len(chut), len(ga), len(ggut)

(20, 22, 29)

In [5]:
query = '가'

In [6]:
chr((ord(query) + 28*21))

'까'

In [7]:
query = '핳'

In [8]:
code = ord(query) - BASE

jongsung = code % 28
print(jongsung, chr(BASE + jongsung))

jungsung = ((code-jongsung) // 28) % 21
print(jungsung, chr(BASE + (jungsung * 28)))

chosung = ((code-jongsung) // 28) // 21
print(chosung, chr(BASE + (chosung * 28 * 21)))

27 갛
0 가
18 하


In [9]:
print(chut[chosung], ga[jungsung], ggut[jongsung])

ㅎ ㅏ ㅎ


# 유니코드 자모음간의 거리 계산
* 없는 부분은 코드(#)로 대체

In [10]:
print(hex(ord('ㅜ') + ))

SyntaxError: invalid syntax (3960724701.py, line 1)

In [ ]:
def segment(ch):
    '''유니코드 글자를 입력받아 초,중,종성에 대한 인덱스를 반환한다'''
    code = ord(ch) - BASE
    jongsung = code % 28
    
    code = code - jongsung
    jungsung = (code // 28) % 21
    
    code = code // 28
    chosung = code // 21
    
    if chosung < 0:
        chosung = -1
    if jongsung < 0:
        jongsung = -1
    return chut[chosung], ga[jungsung], ggut[jongsung]

In [ ]:
segment('갛')

In [ ]:
segment('고')

# 유니코드 자모음간의 거리 계산
* 최장부분수열(LCS)을 계산해서 전체 길이와의 비율로 나타낸다.

In [ ]:
import difflib

def diff(word1, word2):
    '''두 유니코드 단어의 거리를 계산하여 차이를 반환한다'''
    L1 = ''.join(reduce(lambda x1,x2: x1+x2, map(segment, word1)))
    L2 = ''.join(reduce(lambda x1,x2: x1+x2, map(segment, word2)))
    differ = difflib.SequenceMatcher(None, L1, L2)
    return differ.ratio()

In [ ]:
print(diff('이불', '이줄'), diff('이불', '입불'), diff('이불', '이놈'))

# 최근 사용자 검색어 가져오기
* 총 5만개의 쿼리를 대상으로 같은 세션 내에서 60초 이내에 사용자의 검색어 목록을 DB에서 가져온다.
* 직전 검색어와 이후 검색어의 diff ratio가 0.7 이상인 검색어들을 저장한다.
* 직전 검색어와 이후 검색어에 대한 각각의 검색결과 개수를 저장한다.
* query_count.csv 는 저장된 결과

In [11]:
import pandas as pd

In [12]:
df2 = pd.read_csv('query_count.csv', index_col='user_q').drop_duplicates()

In [13]:
df2[:5]

,later_q,user_q_count,later_q_count
user_q,,,
경동나비엔,경동나비엔온수매트,369,93632
락엔락,락앤락,1,7066
경동 나비엔,경동 나비엔,361,361
막대걸레,올터치막대걸레,4578,42378
웰퍼스온수매트,온수매트,93267,93264


# 사용자 검색 정보를 이용하여 오타 찾아내기
* 먼저 검색한 결과(오타키워드)가 10개 미만이면서 후에 검색한 결과(정상키워드)에 비해 개수가 현저하게 적은 검색어를 출력

In [14]:
for index in df2.index:
    row = df2.loc[index]
    row_type = type(row)
    if pd.DataFrame == row_type:
        if row[(row.user_q_count < 10) | 
               ((row.user_q_count / row.later_q_count > 0.3) & (row.user_q_count / row.later_q_count < 1.0))
              ].any().user_q_count:
            print(index, '=>', ','.join(row.later_q.values))
    elif pd.Series == row_type:
        if row.user_q_count < 10 and row.later_q_count > 10:
            print(index, '=>', row.later_q)

락엔락 => 락앤락
식품건조기 => 리큅 식품건조기,리큅식품건조기
뷔아느레 => 비아느레
온수매트 => 일월온수매트,경동 온수매트,온수매트1 1,웰퍼스온수매트,경동온수매트,온열매트,온수매트 경동
펏길 => 퍼실
숀리] => 숀리
전기매트 => 전기카페트,전기매트특대
이연복 탕수육 => 이연복 동파육,이연복탕수육
온수매트 => 일월온수매트,경동 온수매트,온수매트1 1,웰퍼스온수매트,경동온수매트,온열매트,온수매트 경동
여성패딩 => 남성패딩,여성팬티,여성숏패딩,네파여성패딩
경동나비엔온수매트 => 경동나비엔온수매트퀸,경동나비엔,나비엔 온수매트,경동나비엔온수매트1 1,경동나비엔온스매트1 1
초극셋 => 초극세사
맨투맨 => 맨투맨티,학생맨투맨
트렉슈트 => 남성트렉수트
김티 => 김치
엠씨엄 => 엠씨엠
온수매트 => 일월온수매트,경동 온수매트,온수매트1 1,웰퍼스온수매트,경동온수매트,온열매트,온수매트 경동
댕만 => 대만
경동나비엔온수매트 => 경동나비엔온수매트퀸,경동나비엔,나비엔 온수매트,경동나비엔온수매트1 1,경동나비엔온스매트1 1
김치냉장고 => 딤채김치냉장고,디오스김치냉장고 
구그 => 구스
견미리 => 견미리팩트
크림하우스 => 크림하우스1 1,크림하우스 yt,크림하우스매트,크림하우스 매트
리클라아너 => 리클라이너
임모복 => 임부복
온수매트 => 일월온수매트,경동 온수매트,온수매트1 1,웰퍼스온수매트,경동온수매트,온열매트,온수매트 경동
이줄 => 이불
침구류 => 침구
블랙야크신발 => 블랙야크기즈,블랙야크키즈
블랙야크신발 => 블랙야크기즈,블랙야크키즈
아이오페 => 아이오페스템셀,아이오페인텐스,아이오페기초,아이오페쿠션
비자르 암막커튼 => 바자르 암막커튼
블랜도 => 블랜더
스마트빔 => sk스마트빔2,sk스마트빔
스마트빔 => sk스마트빔2,sk스마트빔
이불세트 => 차려이불세트,이불세트k,이불패드
크로스백 => 남자크로스백,미니크로스백
s9041 => 필립스 s9041
여항 => 여행
블래박스 => 블랙박스
경동나비엔온수매트 => 경동나비엔온수매트